In [45]:
# !pip install pytorch-lightning

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path

import os
import random

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from collections import Counter
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

import pytorch_lightning as pl

from tqdm import tqdm_notebook as tqdm

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
train = pd.read_csv('../dataset/경진대회/train_data_dataframe.csv')
test = pd.read_csv('../dataset/경진대회/test_data_dataframe.csv')

In [5]:
train['text'] =  train['context'] + train['title'] + train['question']
test['text'] = test['context'] + test['title']  + test['question']

In [49]:
train.head(2)

,content_id,title,paragraph_id,context,question_id,question,answer_text,answer_start,is_impossible,text
0,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_RrlDreD2hE,"이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...",QUES_CHGHl0CHHK,경쟁 상대국을 연구하고 경쟁 상대국을 앞설 수 있는 방법을 연구하기 위해서 더욱 활...,기술경쟁력,603.0,False,"이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재..."
1,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_s5sSaKdCH3,현위치와 관련된 평가부문은 IMD 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,QUES_ncJEA1ANCe,IMD평가에 포함되었던 기술관리부분의 한국의 순위는 평균 몇 위야,평균 32위,420.0,False,현위치와 관련된 평가부문은 IMD 평가에서나 종합과학기술력의 평가에서 도 포함된 부...


In [6]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [7]:
train['uncased_text'] = train['text'].apply(lambda x: x.lower())
test['uncased_text'] = test['text'].apply(lambda x: x.lower())


In [8]:
train.isnull().sum()

content_id          0
title               0
paragraph_id        0
context             0
question_id         0
question            0
answer_text      9305
answer_start     9305
is_impossible       0
text                0
uncased_text        0
dtype: int64

In [9]:
# Tokenize
train['tokenized_text'] = train['uncased_text'].apply(tokenizer.tokenize)
test['tokenized_text'] = test['uncased_text'].apply(tokenizer.tokenize)

train = train.dropna()
train['tokenized_selected_text'] = train['answer_text'].apply(tokenizer.tokenize)

In [10]:
train.head()

,content_id,title,paragraph_id,context,question_id,question,answer_text,answer_start,is_impossible,text,uncased_text,tokenized_text,tokenized_selected_text
0,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_RrlDreD2hE,"이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...",QUES_CHGHl0CHHK,경쟁 상대국을 연구하고 경쟁 상대국을 앞설 수 있는 방법을 연구하기 위해서 더욱 활...,기술경쟁력,603.0,False,"이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...","이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...","[ᄋ, ##ᅵ, ᄀ, ##ᅳ, ##ᆯ, ##ᄋ, ##ᅦ, ##ᄉ, ##ᅥ, ##ᄂ,...","[ᄀ, ##ᅵ, ##ᄉ, ##ᅮ, ##ᆯ, ##ᄀ, ##ᅧ, ##ᆼ, ##ᄌ, ##..."
1,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_s5sSaKdCH3,현위치와 관련된 평가부문은 IMD 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,QUES_ncJEA1ANCe,IMD평가에 포함되었던 기술관리부분의 한국의 순위는 평균 몇 위야,평균 32위,420.0,False,현위치와 관련된 평가부문은 IMD 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,현위치와 관련된 평가부문은 imd 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,"[[UNK], [UNK], ᄑ, ##ᅧ, ##ᆼ, ##ᄀ, ##ᅡ, ##ᄇ, ##ᅮ...","[ᄑ, ##ᅧ, ##ᆼ, ##ᄀ, ##ᅲ, ##ᆫ, [UNK]]"
3,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_pP8k4n8yg4,IMD의 평가나 종합과학기술력 평가는 궁극적으로는 국가간 순위를 매기기 위한 평가이...,QUES_GnQIVxXt70,기술무역액 등과 같이 가용할 수 있는 통계 수치에 전적으로 의존하는 평가가 뭐야,종합과학기술력 평가,124.0,False,IMD의 평가나 종합과학기술력 평가는 궁극적으로는 국가간 순위를 매기기 위한 평가이...,imd의 평가나 종합과학기술력 평가는 궁극적으로는 국가간 순위를 매기기 위한 평가이...,"[im, ##d, ##ᄋ, ##ᅴ, ᄑ, ##ᅧ, ##ᆼ, ##ᄀ, ##ᅡ, ##ᄂ...","[ᄌ, ##ᅩ, ##ᆼ, ##ᄒ, ##ᅡ, ##ᆸ, ##ᄀ, ##ᅪ, ##ᄒ, ##..."
4,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_7b3M3mKZbb,"기업경영, 산학연 협력, 교육과 훈련, 세계화, 정부의 행정효율 그 어느 부문도 한...",QUES_RcrMURMuhd,창의력을 저해하는 인사제도 요인은 뭐야,연공서열제,120.0,False,"기업경영, 산학연 협력, 교육과 훈련, 세계화, 정부의 행정효율 그 어느 부문도 한...","기업경영, 산학연 협력, 교육과 훈련, 세계화, 정부의 행정효율 그 어느 부문도 한...","[ᄀ, ##ᅵ, ##ᄋ, ##ᅥ, ##ᆸ, ##ᄀ, ##ᅧ, ##ᆼ, ##ᄋ, ##...","[ᄋ, ##ᅧ, ##ᆫ, ##ᄀ, ##ᅩ, ##ᆼ, ##ᄉ, ##ᅥ, ##ᄋ, ##..."
5,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_Ldm3nIToPd,"IMD의 기술경쟁력 평가가 1990년 중반 이후부터는 폭 넓게 인용되고 있지만, 그...",QUES_OhH70koIzZ,과학기술의 성과로 나타나는 기술집약제품이 수출에서 차지하는 비중 등을 바탕으로 하여...,산출지표,348.0,False,"IMD의 기술경쟁력 평가가 1990년 중반 이후부터는 폭 넓게 인용되고 있지만, 그...","imd의 기술경쟁력 평가가 1990년 중반 이후부터는 폭 넓게 인용되고 있지만, 그...","[im, ##d, ##ᄋ, ##ᅴ, ᄀ, ##ᅵ, ##ᄉ, ##ᅮ, ##ᆯ, ##ᄀ...","[ᄉ, ##ᅡ, ##ᆫ, ##ᄎ, ##ᅮ, ##ᆯ, ##ᄌ, ##ᅵ, ##ᄑ, ##ᅭ]"


In [11]:
# Filter train data
start_position_candidates = []
end_position_candidates = []
train['select_length'] = train['tokenized_selected_text'].map(len)

for i in tqdm(range(len(train))):
    start_position_candidate = [j for j, tok in enumerate(train['tokenized_text'].iloc[i]) if tok == train['tokenized_selected_text'].iloc[i][0]]
    end_position_candidate = [j for j, tok in enumerate(train['tokenized_text'].iloc[i]) if tok == train['tokenized_selected_text'].iloc[i][-1]]

    start_position_candidate = [idx for idx in start_position_candidate if idx + train['select_length'].iloc[i] - 1 in end_position_candidate]
    end_position_candidate = [idx for idx in end_position_candidate if idx - train['select_length'].iloc[i] + 1 in start_position_candidate]

    start_position_candidates.append(start_position_candidate)
    end_position_candidates.append(end_position_candidate)

/tmp/ipykernel_3883803/140825938.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(train))):


  0%|          | 0/28024 [00:00<?, ?it/s]

In [12]:
start_position_candidates = [l[0] if len(l) > 0 else -1 for l in start_position_candidates]
end_position_candidates = [l[0] if len(l) > 0 else -1 for l in end_position_candidates]

In [13]:
train['start_position'] = start_position_candidates
train['end_position'] = end_position_candidates
test['start_position'] = -1
test['end_position'] = -1

In [14]:
train

,content_id,title,paragraph_id,context,question_id,question,answer_text,answer_start,is_impossible,text,uncased_text,tokenized_text,tokenized_selected_text,select_length,start_position,end_position
0,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_RrlDreD2hE,"이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...",QUES_CHGHl0CHHK,경쟁 상대국을 연구하고 경쟁 상대국을 앞설 수 있는 방법을 연구하기 위해서 더욱 활...,기술경쟁력,603.0,False,"이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...","이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...","[ᄋ, ##ᅵ, ᄀ, ##ᅳ, ##ᆯ, ##ᄋ, ##ᅦ, ##ᄉ, ##ᅥ, ##ᄂ,...","[ᄀ, ##ᅵ, ##ᄉ, ##ᅮ, ##ᆯ, ##ᄀ, ##ᅧ, ##ᆼ, ##ᄌ, ##...",14,20,33
1,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_s5sSaKdCH3,현위치와 관련된 평가부문은 IMD 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,QUES_ncJEA1ANCe,IMD평가에 포함되었던 기술관리부분의 한국의 순위는 평균 몇 위야,평균 32위,420.0,False,현위치와 관련된 평가부문은 IMD 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,현위치와 관련된 평가부문은 imd 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,"[[UNK], [UNK], ᄑ, ##ᅧ, ##ᆼ, ##ᄀ, ##ᅡ, ##ᄇ, ##ᅮ...","[ᄑ, ##ᅧ, ##ᆼ, ##ᄀ, ##ᅲ, ##ᆫ, [UNK]]",7,653,659
3,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_pP8k4n8yg4,IMD의 평가나 종합과학기술력 평가는 궁극적으로는 국가간 순위를 매기기 위한 평가이...,QUES_GnQIVxXt70,기술무역액 등과 같이 가용할 수 있는 통계 수치에 전적으로 의존하는 평가가 뭐야,종합과학기술력 평가,124.0,False,IMD의 평가나 종합과학기술력 평가는 궁극적으로는 국가간 순위를 매기기 위한 평가이...,imd의 평가나 종합과학기술력 평가는 궁극적으로는 국가간 순위를 매기기 위한 평가이...,"[im, ##d, ##ᄋ, ##ᅴ, ᄑ, ##ᅧ, ##ᆼ, ##ᄀ, ##ᅡ, ##ᄂ...","[ᄌ, ##ᅩ, ##ᆼ, ##ᄒ, ##ᅡ, ##ᆸ, ##ᄀ, ##ᅪ, ##ᄒ, ##...",24,11,34
4,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_7b3M3mKZbb,"기업경영, 산학연 협력, 교육과 훈련, 세계화, 정부의 행정효율 그 어느 부문도 한...",QUES_RcrMURMuhd,창의력을 저해하는 인사제도 요인은 뭐야,연공서열제,120.0,False,"기업경영, 산학연 협력, 교육과 훈련, 세계화, 정부의 행정효율 그 어느 부문도 한...","기업경영, 산학연 협력, 교육과 훈련, 세계화, 정부의 행정효율 그 어느 부문도 한...","[ᄀ, ##ᅵ, ##ᄋ, ##ᅥ, ##ᆸ, ##ᄀ, ##ᅧ, ##ᆼ, ##ᄋ, ##...","[ᄋ, ##ᅧ, ##ᆫ, ##ᄀ, ##ᅩ, ##ᆼ, ##ᄉ, ##ᅥ, ##ᄋ, ##...",13,173,185
5,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_Ldm3nIToPd,"IMD의 기술경쟁력 평가가 1990년 중반 이후부터는 폭 넓게 인용되고 있지만, 그...",QUES_OhH70koIzZ,과학기술의 성과로 나타나는 기술집약제품이 수출에서 차지하는 비중 등을 바탕으로 하여...,산출지표,348.0,False,"IMD의 기술경쟁력 평가가 1990년 중반 이후부터는 폭 넓게 인용되고 있지만, 그...","imd의 기술경쟁력 평가가 1990년 중반 이후부터는 폭 넓게 인용되고 있지만, 그...","[im, ##d, ##ᄋ, ##ᅴ, ᄀ, ##ᅵ, ##ᄉ, ##ᅮ, ##ᆯ, ##ᄀ...","[ᄉ, ##ᅡ, ##ᆫ, ##ᄎ, ##ᅮ, ##ᆯ, ##ᄌ, ##ᅵ, ##ᄑ, ##ᅭ]",10,160,169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37320,CNTS_2129305745,국역 금대전책,PARS_cFgFiF9icI,"실제 측량해 본 것으로 말씀드리면, 종동(宗動)이 가장 높고, 항성(恒星) 토성(土...",QUES_CvNv5UjUIR,우주 가장 아래에서 금성과 수성 외에 항성까지 서로 가릴 때 모든 것을 먹는 것은 뭐지,달,361.0,False,"실제 측량해 본 것으로 말씀드리면, 종동(宗動)이 가장 높고, 항성(恒星) 토성(土...","실제 측량해 본 것으로 말씀드리면, 종동(宗動)이 가장 높고, 항성(恒星) 토성(土...","[ᄉ, ##ᅵ, ##ᆯ, ##ᄌ, ##ᅦ, [UNK], ᄇ, ##ᅩ, ##ᆫ, [U...","[ᄃ, ##ᅡ, ##ᆯ]",3,161,163
37321,CNTS_2129305745,국역 금대전책,PARS_R9RLmyTzUy,"신이 생각하기에, 땅은 계란의 노른자와 같고, 하늘은 계란의 껍질과 같으며, 경도(...",QUES_MrMnyucG4Y,사람이 땅바닥에 있으면서 북쪽으로 이동을 얼마나 해야 북극의 고도가 1도 높아질까,250리,184.0,False,"신이 생각하기에, 땅은 계란의 노른자와 같고, 하늘은 계란의 껍질과 같으며, 경도(...","신이 생각하기에, 땅은 계란의 노른자와 같고, 하늘은 계란의 껍질과 같으며, 경도(...","[ᄉ, ##ᅵ, ##ᆫ, ##ᄋ, ##ᅵ, ᄉ, ##ᅢ, ##ᆼ, ##ᄀ, ##ᅡ,...","[250, ##ᄅ, ##ᅵ]",3,214,216
37324,CNTS_2129305745,국역 금대전책,PARS_ma0DB2fJSk,｢천체론｣은 정조가 하늘의 작용이 민사(民事)와 관계되므로 국가의 다스림에서 천문에...,QUES_XM4RUoboD7,역상을 알지 못함을 분개하면서 이에 대한 견해를 토로하도록 하는 책문을 정조가 내놓...,천체론,1.0,False,｢천체론｣은 정조가 하늘의 작용이 민사(民事)와 관계되므로 국가의 다스림에서 천문에...,｢천체론｣은 정조가 하늘의 작용이 민사(民事)와 관계되므로 국가의 다스림에서 천문에...,"[[UNK], ᄎ, ##ᅥ, ##ᆫ, ##ᄎ, ##ᅦ, ##ᄅ, ##ᅩ, ##ᆫ, ...","[ᄎ, ##ᅥ, ##ᆫ, ##ᄎ, ##ᅦ, ##ᄅ, ##ᅩ, ##ᆫ]",8,1,8
37325,CNTS_2129305745,국역 금대전책,PARS_wGlvzy6QkR,｢문체론｣은 국왕 정조가 “문장은 한 시대의 체제가 있어서 세상의 도와 함께 부침한...,QUES_YsPrUyg1xE,사대부들이 정학을 기피하면서 기교에 기반한 글쓰기를 하자 문체반정을 추진한 왕은 누구야,정조,244.0,False,｢문체론｣은 국왕 정조가 “문장은 한 시대의 체제가 있어서 세상의 도와 함께 부침한...,｢문체론｣은 국왕 정조가 “문장은 한 시대의 체제가 있어서 세상의 도와 함께 부침한...,"[[UNK], ᄆ, ##ᅮ, ##ᆫ, ##ᄎ, ##ᅦ, ##ᄅ, ##ᅩ, ##ᆫ, ...","[ᄌ, ##ᅥ, ##ᆼ, ##ᄌ, ##ᅩ]",5,19,23


In [15]:
train.to_csv('../dataset/경진대회/process_train.csv')
test.to_csv('../dataset/경진대회/process_test.csv')

In [16]:
df_train, df_val = train_test_split(train, train_size=0.8)

In [18]:
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a